# N-gramas

In [1]:
import nltk, pickle
from collections import Counter
from tqdm import tqdm

Aca estaba generalizando las funciones ya existentes, pero calculan los gramas con base en todo el texto plano, y no por oración. Lo dejo comentado por si acaso, aunque hice otra que sí los calcula por oración. Creo que así tiene más sentido.

In [2]:
# def custom_tokenizer(text):
#     import re
#     """
#     Tokeniza un texto personalizado (para los casos de <s>, </s> y <UNK>).

#     Args:
#     text: El texto a tokenizar.

#     Returns:
#     Una lista de tokens.
#     """
#     tokens = re.findall(r"<[^>]+>|[\w']+|[.,!?;]", text)
#     return tokens

# def calculate_ngram_probabilities(corpus: str, n: int) -> Dict[Tuple[str, ...], float]:
#     """
#     Calcula las probabilidades de n-gramas con suavizado de Laplace.

#     Args:
#         corpus: Lista de palabras.
#         n: Tamaño del n-grama (1 para unigrama, 2 para bigrama, etc.).

#     Returns:
#         dict: Un diccionario con las probabilidades de n-gramas.
#     """    
#     tokens = custom_tokenizer(corpus)
#     # Unigrama
#     if n == 1:
#         unigrams = Counter(tokens)
#         V = len(unigrams)  # Tamaño del vocabulario
#         total_words_in_corpus = sum(unigrams.values())
        
#         # Suavizado de Laplace para unigramas
#         probabilities = {
#             (word,): (count + 1) / (total_words_in_corpus + V)
#             for word, count in unigrams.items()
#         }
#         return probabilities

#     # Para n-gramas donde n > 1
#     ngrams_list = list(nltk.ngrams(tokens, n))
#     ngrams = Counter(ngrams_list)

#     # Para (n-1)-gramas (bigramas para trigramas)
#     n_minus_1_grams_list = list(nltk.ngrams(tokens, n - 1))
#     n_minus_1_grams_count = Counter(n_minus_1_grams_list)

#     V = len(set(tokens))  # Tamaño del vocabulario

#     # Suavizado de Laplace para n-gramas
#     probabilities = {
#         ngram: (count + 1) / (n_minus_1_grams_count[ngram[:-1]] + V)  # ngram[:-1] es el (n-1)-grama
#         for ngram, count in ngrams.items()
#     }

#     return probabilities

In [3]:
def read_and_tokenize(file_path):
    """
    Lee un archivo de texto, tokeniza las oraciones y retorna una lista de listas de tokens.
    
    Args:
        file_path: Ruta al archivo de texto.
        
    Returns:
        Lista de listas de tokens (cada oración tokenizada).
    """
    with open(file_path, 'r') as f:
        text = f.read()

    sentences = text.strip().split('\n')
    tokenized_sentences = [sentence.split() for sentence in sentences]
    return tokenized_sentences


def build_ngram_counts(tokenized_sentences, n):
    """
    Cuenta los n-gramas de las oraciones tokenizadas usando nltk
    
    Args:
        tokenized_sentences: Lista de listas de tokens.
        n: Tamaño del n-grama.
        
    Returns:
        Un diccionario con los conteos de n-gramas.
    """
    ngram_counts = Counter()
    
    for sentence in tokenized_sentences:
        ngrams = list(nltk.ngrams(sentence, n))
        ngram_counts.update(ngrams)
    
    return ngram_counts


def calculate_ngram_probabilities(tokenized_sentences, n):
    """
    Calcula las probabilidades de n-gramas con suavizado de Laplace.
    
    Args:
        tokenized_sentences: Lista de listas de tokens.
        n: Tamaño del n-grama.
        
    Returns:
        Diccionario con las probabilidades de los n-gramas.
    """
    # Cuenta los n-gramas y de ser necesario los (n-1)-gramas, para bigramas y trigramas.
    ngram_counts = build_ngram_counts(tokenized_sentences, n)
    n_minus_1_gram_counts = build_ngram_counts(tokenized_sentences, n-1) if n > 1 else None

    # Tamaño del vocabulario
    vocabulary = set(token for sentence in tokenized_sentences for token in sentence)
    vocab_size = len(vocabulary)

    # Diccionario que representará nuestro modelo
    ngram_probabilities = {}

    for ngram, count in tqdm(ngram_counts.items(), desc=f"Calculando {n}-gramas"):
        if n == 1:
            # Para unigramas el conteo es sencillo
            total_count = sum(ngram_counts.values())
            ngram_probabilities[ngram] = (count + 1) / (total_count + vocab_size)
        else:
            # Para bigramas y trigramas hay que tener en cuenta el contexto
            priori = ngram[:-1]
            priori_count = n_minus_1_gram_counts[priori]
            ngram_probabilities[ngram] = (count + 1) / (priori_count + vocab_size)
    
    return ngram_probabilities


In [4]:
tokenized_sentences = read_and_tokenize('20N_training.txt')

unigram_probs = calculate_ngram_probabilities(tokenized_sentences, 1)
bigram_probs = calculate_ngram_probabilities(tokenized_sentences, 2)
trigram_probs = calculate_ngram_probabilities(tokenized_sentences, 3)

Calculando 3-gramas: 100%|██████████| 2002056/2002056 [00:01<00:00, 1353946.03it/s]


In [5]:
print("Unigrams:")
for unigram, prob in list(unigram_probs.items())[:5]:
    print(f"{unigram}: {prob}")

print("\nBigrams:")
for bigram, prob in list(bigram_probs.items())[:5]:
    print(f"{bigram}: {prob}")

print("\nTrigrams:")
for trigram, prob in list(trigram_probs.items())[:5]:
    print(f"{trigram}: {prob}")

Unigrams:
('<s>',): 0.04523324268707575
('does',): 0.0012653016987533416
('anyone',): 0.0007652465680866085
('have',): 0.005621682315234096
('a',): 0.018972472517538957

Bigrams:
('<s>', 'does'): 0.0036281394694836237
('does', 'anyone'): 0.011137933141844405
('anyone', 'have'): 0.004848006316620608
('have', 'a'): 0.03611488176037722
('a', 'listing'): 0.00013497194004404348

Trigrams:
('<s>', 'does', 'anyone'): 0.008182205752139836
('does', 'anyone', 'have'): 0.0033717659829931414
('anyone', 'have', 'a'): 0.0010268810971065141
('have', 'a', 'listing'): 3.1694214221193923e-05
('a', 'listing', 'of'): 0.0002496297159213833


In [6]:
def save_ngram_model(ngram_probabilities, filename):
    with open(filename, 'wb') as file:
        pickle.dump(ngram_probabilities, file)

def load_ngram_model(filename):
    with open(filename, 'rb') as file:
        ngram_probabilities = pickle.load(file)
    return ngram_probabilities

Prefiero usar el formato `pickle` para guardar cosas en python.

In [7]:
save_ngram_model(unigram_probs, "20N_bigrams.pkl")
save_ngram_model(bigram_probs, "20N_bigrams.pkl")
save_ngram_model(trigram_probs, "20N_trigrams.pkl")

In [8]:
bigram_probs[("</s>", "<s>")] # Esto antes si se permitía, creo que no tiene sentido.

KeyError: ('</s>', '<s>')

### Generando texto

In [9]:
def generate_text(model, start_sentence, max_length=20, n=1):
    """
    Genera texto usando el modelo n-grama basado en el n-grama de mayor probabilidad.

    Args:
        model: Diccionario de n-gramas con sus probabilidades.
        start_sentence: Frase de inicio como string.
        max_length: Longitud máxima de la oración generada.
        n: Tamaño del n-grama (ej. 1 para unigramas).

    Returns:
        string: Oración generada.
    """
    sentence = start_sentence.split()

    if n == 1:
        # Para unigramas, seleccionamos el token con la mayor probabilidad en cada paso
        # En realidad no tiene sentido generar con unigramas
        for _ in range(max_length):
            next_token = max(model, key=model.get)
            if next_token == '</s>':
                break
            sentence.append(next_token[0])
    else:
        # Para n-gramas donde n > 1
        for _ in range(max_length):
            if len(sentence) < n-1:
                context = tuple(sentence)  # Usa toda la oración disponible
            else:
                context = tuple(sentence[-(n-1):])  # Toma los últimos n-1 tokens como contexto
            
            # Buscar las posibles continuaciones en el modelo
            possible_ngrams = [ngram for ngram in model if ngram[:-1] == context]
            
            if not possible_ngrams:
                break
            
            next_token = max(possible_ngrams, key=lambda ngram: model[ngram])[-1]
            
            if next_token == '</s>':
                break
            sentence.append(next_token)
    
    return ' '.join(sentence)

In [10]:
generate_text(trigram_probs, "<s> hi", max_length= 50, n = 3)

'<s> hi i am not sure if this is a good idea to have a NUM a NUM a NUM a NUM a NUM a NUM a NUM a NUM a NUM a NUM a NUM a NUM a NUM a NUM a NUM a NUM a NUM a NUM a NUM'

In [11]:
generate_text(unigram_probs, "<s>", max_length= 50, n = 1)
# Obviamente generar con unigramas no tiene sentido.

'<s> NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM NUM'

In [12]:
import math

def calculate_perplexity_log(model, test_sentences, n):
    total_log_prob = 0
    total_ngrams = 0
    
    for sentence in test_sentences:
        sentence_log_prob = 0
        ngram_count = 0
        
        n_grams = list(nltk.ngrams(sentence,n))

        for ngram in n_grams:            
            if ngram in model:
                prob = model[ngram]
            else:
                prob = 1e-10
            
            sentence_log_prob += math.log(prob)
            ngram_count += 1
        
        total_log_prob += sentence_log_prob
        total_ngrams += ngram_count
    
    avg_log_prob = total_log_prob / total_ngrams
    perplexity = math.exp(-avg_log_prob)
    
    return perplexity

In [13]:
test_sentences = read_and_tokenize("20N_testing.txt")

In [14]:
perplexity = calculate_perplexity_log(trigram_probs, test_sentences, n=3)
print(f'Perplejidad del modelo de trigramas: {perplexity}')

Perplejidad del modelo de trigramas: 1421914.8987478605


In [15]:
perplexity = calculate_perplexity_log(bigram_probs, test_sentences, n=2) 
print(f'Perplejidad del modelo de bigramas: {perplexity}')

Perplejidad del modelo de bigramas: 11099.948099774296


In [16]:
perplexity = calculate_perplexity_log(unigram_probs, test_sentences, n=1)
print(f'Perplejidad del modelo de unigramas: {perplexity}')

Perplejidad del modelo de unigramas: 957.6217625396342
